## Generate class-slice indexing table for experiments


### Overview

This is for experiment setting up for simulating few-shot image segmentation scenarios

Input: pre-processed images and their ground-truth labels

Output: a `json` file for class-slice indexing

In [17]:
%reset
%load_ext autoreload
%autoreload 2
import numpy as np
import os
import glob
import SimpleITK as sitk
import niftiio as nio
import sys
import json

Once deleted, variables cannot be recovered. Proceed (y/[n])? y
The autoreload extension is already loaded. To reload it, use:
  %reload_ext autoreload


In [18]:
IMG_BNAME="../data/chaos_MR_train_T2_histv3/image_*.nii.gz"
SEG_BNAME="../data/chaos_MR_train_T2_histv3/label_*.nii.gz"

In [19]:
imgs = glob.glob(IMG_BNAME)
segs = glob.glob(SEG_BNAME)
imgs = [ fid for fid in sorted(imgs, key = lambda x: int(x.split("_")[-1].split(".nii.gz")[0])  ) ]
segs = [ fid for fid in sorted(segs, key = lambda x: int(x.split("_")[-1].split(".nii.gz")[0])  ) ]


In [20]:
imgs

['../data/chaos_MR_train_T2_histv3/image_1.nii.gz',
 '../data/chaos_MR_train_T2_histv3/image_2.nii.gz',
 '../data/chaos_MR_train_T2_histv3/image_3.nii.gz',
 '../data/chaos_MR_train_T2_histv3/image_5.nii.gz',
 '../data/chaos_MR_train_T2_histv3/image_8.nii.gz',
 '../data/chaos_MR_train_T2_histv3/image_10.nii.gz',
 '../data/chaos_MR_train_T2_histv3/image_13.nii.gz',
 '../data/chaos_MR_train_T2_histv3/image_15.nii.gz',
 '../data/chaos_MR_train_T2_histv3/image_19.nii.gz',
 '../data/chaos_MR_train_T2_histv3/image_20.nii.gz',
 '../data/chaos_MR_train_T2_histv3/image_21.nii.gz',
 '../data/chaos_MR_train_T2_histv3/image_22.nii.gz',
 '../data/chaos_MR_train_T2_histv3/image_31.nii.gz',
 '../data/chaos_MR_train_T2_histv3/image_32.nii.gz',
 '../data/chaos_MR_train_T2_histv3/image_33.nii.gz',
 '../data/chaos_MR_train_T2_histv3/image_34.nii.gz',
 '../data/chaos_MR_train_T2_histv3/image_36.nii.gz',
 '../data/chaos_MR_train_T2_histv3/image_37.nii.gz',
 '../data/chaos_MR_train_T2_histv3/image_38.nii.gz'

In [22]:
segs

['../data/chaos_MR_train_T2_histv3/label_1.nii.gz',
 '../data/chaos_MR_train_T2_histv3/label_2.nii.gz',
 '../data/chaos_MR_train_T2_histv3/label_3.nii.gz',
 '../data/chaos_MR_train_T2_histv3/label_5.nii.gz',
 '../data/chaos_MR_train_T2_histv3/label_8.nii.gz',
 '../data/chaos_MR_train_T2_histv3/label_10.nii.gz',
 '../data/chaos_MR_train_T2_histv3/label_13.nii.gz',
 '../data/chaos_MR_train_T2_histv3/label_15.nii.gz',
 '../data/chaos_MR_train_T2_histv3/label_19.nii.gz',
 '../data/chaos_MR_train_T2_histv3/label_20.nii.gz',
 '../data/chaos_MR_train_T2_histv3/label_21.nii.gz',
 '../data/chaos_MR_train_T2_histv3/label_22.nii.gz',
 '../data/chaos_MR_train_T2_histv3/label_31.nii.gz',
 '../data/chaos_MR_train_T2_histv3/label_32.nii.gz',
 '../data/chaos_MR_train_T2_histv3/label_33.nii.gz',
 '../data/chaos_MR_train_T2_histv3/label_34.nii.gz',
 '../data/chaos_MR_train_T2_histv3/label_36.nii.gz',
 '../data/chaos_MR_train_T2_histv3/label_37.nii.gz',
 '../data/chaos_MR_train_T2_histv3/label_38.nii.gz'

In [26]:
classmap = {}
LABEL_NAME = ["BG", "LIVER", "RK", "LK", "SPLEEN"]     


MIN_TP = 1 # minimum number of positive label pixels to be recorded. Use >100 when training with manual annotations for more stable training

fid = f'../data/chaos_MR_train_T2_histv3/classmap_{MIN_TP}.json' # name of the output file. 
for _lb in LABEL_NAME:
    classmap[_lb] = {}
    for _sid in segs:
        pid = _sid.split("_")[-1].split(".nii.gz")[0]
        classmap[_lb][pid] = []

for seg in segs:
    pid = seg.split("_")[-1].split(".nii.gz")[0]
    lb_vol = nio.read_nii_bysitk(seg)
    n_slice = lb_vol.shape[0]
    for slc in range(n_slice):
        for cls in range(len(LABEL_NAME)):
            if cls in lb_vol[slc, ...]:
                if np.sum( lb_vol[slc, ...]) >= MIN_TP:
                    classmap[LABEL_NAME[cls]][str(pid)].append(slc)
    print(f'pid {str(pid)} finished!')
    
with open(fid, 'w') as fopen:
    json.dump(classmap, fopen)
    fopen.close()  
    

pid 1 finished!
pid 2 finished!
pid 3 finished!
pid 5 finished!
pid 8 finished!
pid 10 finished!
pid 13 finished!
pid 15 finished!
pid 19 finished!
pid 20 finished!
pid 21 finished!
pid 22 finished!
pid 31 finished!
pid 32 finished!
pid 33 finished!
pid 34 finished!
pid 36 finished!
pid 37 finished!
pid 38 finished!
pid 39 finished!


In [20]:
with open(fid, 'w') as fopen:
    json.dump(classmap, fopen)
    fopen.close()